# Hands On Instrument Data Selection and Binning

Please refer to [this notebook](https://github.com/arqus-gamma/arqus_school_2022/blob/main/hands_on/1_read_data/Read_data_with_astropy_hands_on.ipynb) for preliminary introduction

## 0. Setup

As usual, we start by installing some standards packages. We try to be minimal and install only those stricly required.
Check package versions. All examples should work with Astropy > 2.0 and Numpy > 1.11

In [ ]:
%matplotlib inline  
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import astropy
from astropy import units as u
from astropy import constants as const
#from astropy.coordinates import SkyCoord
# from astropy.time import Time
from astropy.table import Table
from astropy.io import fits


print('numpy:', np.__version__)
print('astropy:', astropy.__version__)

However, in most cases, remember that for analyzing specific instruments' data, often additional packages are to be installed, that are somewhat specific of the instrument. In many cases, they are tailored functions

In [ ]:
import gammapy
from gammapy.data import Observation, Observations, DataStore
from gammapy.makers import SafeMaskMaker, MapDatasetMaker, SpectrumDatasetMaker, ReflectedRegionsBackgroundMaker, WobbleRegionsFinder

# dataset modules
from gammapy.datasets import (
    MapDataset, 
    MapDatasetOnOff, 
    MapDatasetEventSampler,
    SpectrumDatasetOnOff,
    SpectrumDataset, 
    Datasets,
    FluxPointsDataset
)

## 1. Select data

In this part we start selecting the data of interest for the further analysis.

Let's first check what's in the data folder

In [ ]:
!ls ../../data/

We see that we have 4 targets, as well the complete list of files. Let's open one target:


In [ ]:
!ls ../../data/crab_nebula/

Great! More than 5 instruments data for our targets. Let's start with MAGIC data:

In [ ]:
!ls ../../data/crab_nebula/magic

So we have two files (run): 05029747 and 05029748. Let's open one of them. 

In [ ]:
#fits_file = fits.open('../../data/crab_nebula/magic/run_05029747_DL3.fits')
fits_file = fits.open('../../data/ngc1275/magic/20170101_05059487_DL3_NGC1275-W0.40+157.fits')

Let's check the header data unit (HDU) by calling `.info()`:

In [ ]:
fits_file.info()

The PRIMARY contains only a short header and no data

In [ ]:
primary = fits_file['PRIMARY'] 
primary.header
#primary.data

Let's look at the EVENT HDU. Here are the events information. First with the header

In [ ]:
events = fits_file['EVENTS'] 
events.header

In [ ]:
events.columns.names

Of all the entries, some are worth discussing

```
OBS_ID  =              5059487 / unique observation identifier                  
TSTART  =         4.361391E+08 / [s] observation start time                     
TSTOP   =         4.361403E+08 / [s] observation end time                       
ONTIME  =                1193. / [s] total good time (sum of GTIs)              
LIVETIME=             1178.607 / [s] total effective time                       
OBS_MODE= 'POINTING'           / observation mode                               
RA_OBJ  =                49.95 / [deg] source RA                                
DEC_OBJ =             41.51167 / [deg] source DEC                               
OBS_MODE= 'Wobble  '           / observation mode                               
RA_PNT  =             49.45417 / [deg] observation pointing RA                  
DEC_PNT =             41.66806 / [deg] observation pointing DEC                 
ALT_PNT =             76.89065 / [deg] average altitude pointing                
AZ_PNT  =            -6.315182 / [deg] average azimuth pointing
```

Let's look at the data now

In [ ]:
events.data

In [ ]:
Table(events.data)

In [ ]:
en  = events.data['ENERGY']
t   = events.data['TIME']
ra  = events.data['RA']
dec = events.data['DEC']
g   = events.data['GAMMANESS']
dx  = events.data['DETX']
dy  = events.data['DETY']

In [ ]:
# Selection
cuts1 = events.data['GAMMANESS'] > 0.95
evt = events.data[cuts1]
print(len(events.data))
print(len(evt))

In [ ]:
ensel = evt['ENERGY']
tsel   = evt['TIME']
rasel     = evt['RA']
decsel    = evt['DEC']
gsel = evt['GAMMANESS']
dxsel   = evt['DETX']
dysel   = evt['DETY']

In [ ]:
# Direction of the events
plt.plot(dec,ra,marker='s', markersize='0.2',linewidth='0',alpha=0.6, label="all")
plt.plot(decsel,rasel,marker='s', markersize='0.4',linewidth='0',label="selected")
plt.xlabel('Dec [deg]')
plt.ylabel('RA [deg]')
plt.legend(loc="best")

In [ ]:
# Direction of the events
plt.plot(dx,dy,marker='s', markersize='0.2',linewidth='0')


In [ ]:
binning = np.linspace(0,1,50)
plt.hist(g,bins=binning,alpha=0.8)
plt.hist(gsel,bins=binning,alpha=0.9)


Everything looks ok. For the sake of the following part we need the EVENTS tables for the moment

In [ ]:
energy = events.data['ENERGY']
time = events.data['TIME']
condizione = energy > 1.05
condtemp = time >  334778850.0339344
event_sel = events.data[condizione*condtemp]
event_sel
gammaness = event_sel['GAMMANESS']
len(gammaness)
#Table(event_sel)
#plt.hist(energy, bins=1000)

## Effective Area

In [ ]:
effarea = fits_file['EFFECTIVE AREA'] 
effarea.header
Table(effarea.data)
enlo = effarea.data['ENERG_LO']
aeff = effarea.data['EFFAREA']
x = enlo[0]
y = aeff[0][0]
x*u.TeV

In [ ]:
plt.loglog(x,y)
plt.xlabel('Energy [TeV]')
plt.ylabel('Area [m^2]')

In [ ]:
binning = np.logspace(-2,1,20)
plt.hist(en,bins=binning,alpha=0.6,log=True)
plt.hist(ensel,bins=binning,alpha=0.8,log=True)
plt.xscale('log')